In [1]:
import pygame
import numpy as np
from keras.models import load_model
import carla
from agents.navigation.basic_agent import BasicAgent
import tensorflow as tf
import math

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


2022-08-19 15:23:52.490140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 15:23:52.490158: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Connect to simulator

In [2]:
client = carla.Client('192.168.5.145', 2000)
client.set_timeout(4.0)
client.load_world('Town02')
traffic_manager = client.get_trafficmanager()
world = client.get_world()
world.set_weather(carla.WeatherParameters.CloudySunset)


Set Synchronous FPS

In [3]:
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)
traffic_manager.set_synchronous_mode(True)

Add Vehicle to right location

In [4]:
start_location = carla.Location(x=-7.530004, y=234.110001, z=0.500000)
end_location = carla.Location(x=166.914505, y=191.770035, z=0.500000)
back_location = carla.Location(x=173.870056, y=105.550011, z=0.500000)
back_transform = carla.Transform(back_location, carla.Rotation(pitch=0.000000, yaw=-180, roll=0.000000))
start_transform = carla.Transform(start_location, carla.Rotation(yaw=90))
end_transform = carla.Transform(start_location, carla.Rotation(yaw=0))
bp_lib = world.get_blueprint_library()
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, start_transform)
world.get_spectator().set_transform(start_transform)

In [5]:
vehicle.set_transform(start_transform)

In [6]:
traffic_manager.ignore_lights_percentage(vehicle, 100)
path_list = [start_location, end_location, back_location, start_location, end_location, back_location, start_location]
traffic_manager.set_path(vehicle, path_list, empty_buffer=True)
vehicle.set_autopilot(True)

In [7]:
print(vehicle.get_control().steer)
world.tick()
print(vehicle.get_control().steer)

-52468318208.0
0.0


In [8]:
# location = vehicle.get_location()
# transform = vehicle.get_transform()
# rotation = transform.rotation
#
# vehicle.set_transform(
#     carla.Transform(
#         carla.Location(x=location.x + 1, y=location.y, z=location.z),
#         carla.Rotation(pitch=rotation.pitch, yaw=rotation.yaw, roll=rotation.roll)))

In [9]:
# while True:
#     world.tick()
#     transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
#                                 vehicle.get_transform().rotation)
#     world.get_spectator().set_transform(transform)
#     # if 160 < vehicle.get_location().x < 170 & 185 < vehicle.get_location().y < 195:
#     #     print('end')
#     #     break
#     #     traffic_manager.set_path(vehicle, [end_location, start_location], empty_buffer=True)
#     # if -12 < vehicle.get_location().x < -2 & 229 < vehicle.get_location().y < 240:
#     #     break
#     #     print('start')
#     #     traffic_manager.set_path(vehicle, [start_location, end_location], empty_buffer=True)
#     # print(traffic_manager.get_next_action(vehicle))
#     print(traffic_manager.get_all_actions(vehicle))



Add agent with destination

In [10]:
opt_dict = {"ignore_traffic_lights": True, "ignore_stop_signs": True}
agent = BasicAgent(vehicle=vehicle, target_speed=15, opt_dict=opt_dict)
agent.set_destination(end_location=end_location, start_location=start_location)

Define Callback for Pygame

In [11]:
# Render object to keep and pass the PyGame surface
class RenderObject(object):
    def __init__(self, width, height):
        init_image = np.random.randint(0, 255, (height, width, 3), dtype='uint8')
        self.surface = pygame.surfarray.make_surface(init_image.swapaxes(0, 1))


# Camera sensor callback, reshapes raw data from camera into 2D RGB and applies to PyGame surface
def front_camera_callback(data, obj, trainer, _agent):
    img = np.reshape(np.copy(data.raw_data), (data.height, data.width, 4))
    img = img[:, :, :3]
    img = img[:, :, ::-1]
    obj.surface = pygame.surfarray.make_surface(img.swapaxes(0, 1))
    # reshaped_image = np.expand_dims(img, axis=0)
    #
    # vel = vehicle.get_velocity()
    # current_speed = (3.6 * math.sqrt(vel.x ** 2 + vel.y ** 2 + vel.z ** 2))
    # speed_limit = vehicle.get_speed_limit()
    #
    # _agent.get_local_planner().run_step(True)
    # high_level = int(_agent.get_local_planner().get_incoming_waypoint_and_direction(0)[1].value)
    # print(high_level)
    #
    # hlc = np.zeros((1, 6))
    # hlc[0][0] = current_speed
    # hlc[0][1] = speed_limit
    # if high_level == 1:
    #     hlc[0][2] = 1
    # elif high_level == 2:
    #     hlc[0][3] = 1
    # elif high_level == 3:
    #     hlc[0][4] = 1
    # else:
    #     hlc[0][5] = 1
    #
    # prediction = trainer.model.predict([reshaped_image, hlc])
    # trainer.control.steer = float(prediction[0][0])
    # trainer.control.throttle = float(prediction[0][1])

Create ModelTrainer

In [12]:
class ModelTrainer(object):
    def __init__(self, path: str):
        self.model = self.model = load_model(path)
        self.control = carla.VehicleControl()


model_trainer = ModelTrainer('CNN_Models/hege_1hour_town02_no_val_speed_limit')

2022-08-19 15:24:02.893617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-19 15:24:02.893803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-19 15:24:02.893847: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-19 15:24:02.893887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-19 15:24:02.893928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Add front camera

In [13]:
# location = vehicle.get_location()
# transform = vehicle.get_transform()
# rotation = transform.rotation
#
# cm_transform = carla.Transform(
#     carla.Location(x=location.x, y=location.y, z=location.z),
#     carla.Rotation(pitch=rotation.pitch, yaw=rotation.yaw + 30, roll=rotation.roll))

front_camera_bp = bp_lib.find('sensor.camera.rgb')
front_camera_bp.set_attribute('image_size_x', '300')
front_camera_bp.set_attribute('image_size_y', '180')
front_camera_transform = carla.Transform(carla.Location(x=1.6, z=1.7),
                                         carla.Rotation(yaw=30))
front_camera = world.spawn_actor(front_camera_bp, front_camera_transform, attach_to=vehicle)

Call pygame callback

In [14]:
image_w = front_camera_bp.get_attribute("image_size_x").as_int()
image_h = front_camera_bp.get_attribute("image_size_y").as_int()
renderObject = RenderObject(image_w, image_h)
front_camera.listen(lambda image: front_camera_callback(image, renderObject, model_trainer, agent))

Initialize pygame display

In [15]:
pygame.init()
display = pygame.display.set_mode((image_w, image_h), pygame.HWSURFACE | pygame.DOUBLEBUF)
display.fill((0, 0, 0))
display.blit(renderObject.surface, (0, 0))
pygame.display.flip()

In [11]:
traffic_manager.ignore_lights_percentage(vehicle, 100)
vehicle.set_autopilot(False)

In [16]:
crashed = False
while not crashed:
    world.tick()
    transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
                                vehicle.get_transform().rotation)
    world.get_spectator().set_transform(transform)
    # vehicle.apply_control(model_trainer.control)
    display.blit(renderObject.surface, (0, 0))
    pygame.display.flip()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            crashed = True